<a href="https://colab.research.google.com/github/NickSlm/ml1/blob/main/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [72]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds

In [73]:
dataset = keras.datasets.imdb.load_data()

In [74]:
(X_train, y_train), (X_test, y_test) = dataset

In [75]:
test_seq = X_train[0][:10]

In [76]:
word_index = keras.datasets.imdb.get_word_index()

In [79]:
id_to_word = dict((id_ + 3,word) for word, id_ in word_index.items())
id_to_word [1] = "[START]"
id_to_word [2] = "[OOV]"
" ".join(id_to_word[id_] for id_ in test_seq)

'[START] this film was just brilliant casting location scenery story'

In [80]:
dataset, info = tfds.load("imdb_reviews", as_supervised=True, with_info=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteP3SRCQ/imdb_reviews-train.tfrecord…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteP3SRCQ/imdb_reviews-test.tfrecord*…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteP3SRCQ/imdb_reviews-unsupervised.t…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [82]:
dataset

{Split('train'): <_PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>,
 Split('test'): <_PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>,
 Split('unsupervised'): <_PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>}

In [89]:
train_size = info.splits["train"].num_examples
test_size = info.splits["test"].num_examples

In [ ]:
def preprocess(X_batch_ y_batch):
  X_batch = tf.strings.substr(X_batch, 0, 300)
  X_batch = tf.strings.regex_replace(X_batch, )

In [101]:
test = "Review: This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting  ...Label: 0 = Negative"
test = tf.strings.substr(test, 0, 300)
# test = tf.strings.regex_replace(test, rb"<br\s*/?>", b" ")
# test = tf.strings.regex_replace(test, b"[^a-zA-Z']", b" ")
test = tf.strings.split(test)
test

<tf.Tensor: shape=(39,), dtype=string, numpy=
array([b'Review:', b'This', b'was', b'an', b'absolutely', b'terrible',
       b'movie.', b"Don't", b'be', b'lured', b'in', b'by', b'Christopher',
       b'Walken', b'or', b'Michael', b'Ironside.', b'Both', b'are',
       b'great', b'actors,', b'but', b'this', b'must', b'simply', b'be',
       b'their', b'worst', b'role', b'in', b'history.', b'Even', b'their',
       b'great', b'acting', b'...Label:', b'0', b'=', b'Negative'],
      dtype=object)>